# Подготовка

## Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

## Загрузка данных

In [3]:
items = pd.read_parquet("./data/transformed/items.par")
events = pd.read_parquet("./data/transformed/events.par")

## Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

In [4]:
event_cutoff_date = pd.to_datetime("2017-08-01").date()
events_train = events.query("started_at < @event_cutoff_date").copy()
events_test = events.query("started_at >= @event_cutoff_date").copy()
users_train = set(events_train["user_id"].unique())
users_test = set(events_test["user_id"].unique())
common_users = set(users_train & users_test)
print(len(users_train), len(users_test), len(common_users)) 

428220 123223 120858


# === Знакомство: "холодный" старт

In [5]:
# cold users
cold_users = set(users_test - users_train)
print(len(cold_users))

2365


In [6]:
from sklearn.preprocessing import MinMaxScaler

top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()

# нормализация пользователей и среднего рейтинга, требуется для их приведения к одному масштабу
scaler = MinMaxScaler()
item_popularity[["users_norm", "avg_rating_norm"]] = scaler.fit_transform(
    item_popularity[["users", "avg_rating"]]
)

# вычисляем popularity_score, как скор популярности со штрафом за низкий рейтинг
item_popularity["popularity_score"] = (
    item_popularity["users_norm"] * item_popularity["avg_rating_norm"]
)

# сортируем по убыванию popularity_score
item_popularity = item_popularity.sort_values("popularity_score", ascending=False)

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.query("avg_rating >= 4").head(100)
top_k_pop_items.head()

,item_id,users,avg_rating,users_norm,avg_rating_norm,popularity_score
32387,18007564,20207,4.321275,0.496596,0.830319,0.412333
32623,18143977,19462,4.290669,0.478287,0.822667,0.393471
2,3,15139,4.706057,0.372042,0.926514,0.344702
30695,16096824,16770,4.301014,0.412126,0.825253,0.340108
1916,15881,13043,4.632447,0.320529,0.908112,0.291076


In [7]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 10):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_score", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_score,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,0.412333,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,0.393471,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,0.344702,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,0.340108,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
4,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,0.291076,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
...,...,...,...,...,...,...,...,...
95,8490112,Laini Taylor,Daughter of Smoke & Bone (Daughter of Smoke & ...,2011,4792,4.080968,0.090694,"{'Fantasy': 11681, 'Young Adult': 7110, 'Roman..."
96,18966819,Pierce Brown,"Golden Son (Red Rising, #2)",2015,4361,4.374914,0.090409,"{'Science Fiction': 2613, 'Fantasy': 1372, 'Sc..."
97,3636,Lois Lowry,"The Giver (The Giver, #1)",2006,4667,4.098564,0.088832,"{'Young Adult': 10993, 'Fiction': 9045, 'Class..."
98,18293427,Gabrielle Zevin,The Storied Life of A.J. Fikry,2014,4674,4.092640,0.088795,"{'Fiction': 3795, 'Contemporary': 1100, 'Writi..."


In [8]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items, on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]]
print(len(cold_user_recs) /  len(cold_users_events_with_recs))

0.19768403639371382


In [9]:

from sklearn.metrics import mean_absolute_error, root_mean_squared_error


rmse = root_mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [10]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

# === Базовые подходы: коллаборативная фильтрация

In [7]:
# ~3-5 minutes
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [10]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [19]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
print(rmse, mae)

RMSE: 0.8289
MAE:  0.6474
0.8288711689059135 0.647437483750257


In [8]:
from surprise import NormalPredictor, accuracy

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)
random_rmse = accuracy.rmse(random_predictions)
random_mae = accuracy.mae(random_predictions)
print(random_rmse, random_mae)

NameError: name 'surprise_test_set' is not defined

In [23]:
(random_mae - mae) / mae * 100

54.72886771309964

### Факультативное задание
Задание. Удалите из events события для редких айтемов — таких, с которыми взаимодействовало менее N пользователей. Возьмите небольшое N, например 2–3 пользователя. Получите рекомендации, посчитайте метрики, оцените, как они изменились. Подумайте, с чем могут быть связаны такие изменения.

In [9]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):
    """
    возвращает n рекомендаций для user_id
    """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'])
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events["user_id"] == user_id]['item_id'])
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    left = [smth for smth in predictions if smth.iid == 31450846]  # 30688013
    print(left)
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"])

get_recommendations_svd(1296647, items, events_test, svd_model)

[Prediction(uid=1296647, iid=31450846, r_ui=None, est=4.752795975597794, details={'was_impossible': False})]


,item_id,score
0,11221285,4.914296
1,22037424,4.908423
2,33353628,4.872179
3,29844341,4.850003
4,17332218,4.839010


## Дополнительная проверка качества рекомендаций

In [31]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations)

user_id: 648870e2d02a8adcfbf23f78045204dd
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
68,Veronica Roth,"Divergent (Divergent, #1)",2014-06-02,2014-06-04,4,"{'Young Adult': 20260, 'Science Fiction-Dystop..."
69,"Gillian Flynn, В. Русанов",Gone Girl,2014-05-27,2014-05-29,5,"{'Fiction': 11773, 'Mystery': 9965, 'Thriller'..."
70,Kathy Reichs,"Death du Jour (Temperance Brennan, #2)",2014-05-24,2014-05-27,4,"{'Mystery': 1206, 'Mystery-Crime': 579, 'Ficti..."
71,Chelsea Cain,"Heartsick (Archie Sheridan & Gretchen Lowell, #1)",2014-05-22,2014-05-22,5,"{'Mystery': 832, 'Thriller': 653, 'Fiction': 4..."
72,"Jussi Adler-Olsen, Lisa Hartford","The Keeper of Lost Causes (Department Q, #1)",2014-05-30,2014-06-02,3,"{'Mystery': 1225, 'Mystery-Crime': 627, 'Ficti..."
73,Gillian Flynn,Dark Places,2014-05-17,2014-05-22,4,"{'Mystery': 4534, 'Fiction': 4055, 'Thriller':..."
74,Audrey Niffenegger,Her Fearful Symmetry,2014-05-05,2014-05-08,2,"{'Fiction': 1984, 'Fantasy': 674, 'Fantasy-Par..."
75,Kathy Reichs,"Déjà Dead (Temperance Brennan, #1)",2014-05-13,2014-05-17,4,"{'Mystery': 2141, 'Fiction': 904, 'Mystery-Cri..."
76,Carolyn Parkhurst,The Dogs of Babel,2014-05-09,2014-05-10,5,"{'Fiction': 522, 'Mystery': 102, 'Animals': 77..."
77,George R.R. Martin,"A Dance with Dragons (A Song of Ice and Fire, #5)",2014-05-04,2014-05-04,5,"{'Fantasy': 22247, 'Fiction': 4512, 'Fantasy-E..."


Рекомендации
[Prediction(uid='648870e2d02a8adcfbf23f78045204dd', iid=30688013, r_ui=None, est=4.9910641738662145, details={'was_impossible': False})]


,item_id,score,author,title,genre_and_votes
0,2199,5,Doris Kearns Goodwin,Team of Rivals: The Political Genius of Abraha...,"{'History': 4174, 'Nonfiction': 2127, 'Biograp..."
1,16255632,5,"David Gaider, Ben Gelinas, Mike Laidlaw, Dave ...",Dragon Age: The World of Thedas Volume 1,"{'Fantasy': 134, 'Games-Video Games': 28, 'Art..."
2,2363958,5,João Guimarães Rosa,Grande Sertão: Veredas,"{'Fiction': 85, 'Classics': 69, 'Cultural-Braz..."
3,22552026,5,Jason Reynolds,Long Way Down,"{'Young Adult': 1871, 'Poetry': 1737, 'Contemp..."
4,29237211,5,"Brian K. Vaughan, Fiona Staples","Saga, Vol. 7 (Saga, #7)","{'Sequential Art-Graphic Novels': 2539, 'Seque..."


## Коллаборативная фильтрация: ALS

In [5]:
from als_model import ALSPredictor
predictor = ALSPredictor(items, events, events_train, events_test)
predictor.fit()

d:\Work\ya.practicum\ml\sprint-4\mle-recsys-start\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Encoding data...
User encoding took 3.90 seconds
Item encoding took 2.17 seconds
User-item matrix construction took 0.02 seconds
Encoding data...
User encoding took 3.78 seconds
Item encoding took 2.07 seconds
User-item matrix construction took 0.02 seconds
Loading ALS model from d:\Work\ya.practicum\ml\sprint-4\mle-recsys-start\data\models\als_model.pkl


43304

In [ ]:
# TODO: Используя get_recommendations_als, напишите код, который позволит для случайного пользователя просмотреть рекомендации в удобном формате: 
# история с именами авторов и названием книг,
# рекомендации с именами авторов и названием книг, seen-признаком (взаимодействовал ли уже пользователь с рекомендованной книгой).
# Проанализируйте, релевантны ли рекомендации имеющейся истории.
user_id = events_test['user_id'].sample().iat[0]
print(f"user_id: {user_id}")
user_history = (
    events_test
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
    .sort_values("started_at", ascending=False)
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)
user_recommendations = predictor.recommend(user_id)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
user_recommendations["seen"] = user_recommendations["item_id"].isin(user_history["item_id"])
display(user_recommendations[["author", "title", "seen", "score", "genre_and_votes"]])

user_id: 6028cc455bfd956c6ad6834ba4b936b4


,author,title,started_at,read_at,rating,genre_and_votes
46,Julie Kagawa,"The Iron Daughter (The Iron Fey, #2)",2014-04-15,2014-04-22,5,"{'Fantasy': 2782, 'Young Adult': 2072, 'Romanc..."
47,Julie Kagawa,"Winter's Passage (Iron Fey, #1.5)",2014-04-13,2014-04-15,5,"{'Fantasy': 937, 'Young Adult': 785, 'Fantasy-..."
49,Julie Kagawa,"The Iron King (The Iron Fey, #1)",2014-04-07,2014-04-13,4,"{'Fantasy': 5446, 'Young Adult': 3802, 'Romanc..."
53,Jessica Sorensen,"The Temptation of Lila and Ethan (The Secret, #3)",2014-04-01,2014-04-07,3,"{'New Adult': 331, 'Romance': 227, 'Contempora..."
54,Jessica Sorensen,"The Forever of Ella and Micha (The Secret, #2)",2014-03-27,2014-04-01,4,"{'New Adult': 475, 'Romance': 399, 'Contempora..."
55,Jessica Sorensen,"The Secret of Ella and Micha (The Secret, #1)",2014-03-21,2014-03-27,5,"{'New Adult': 1108, 'Romance': 1046, 'Contempo..."
50,Susan Ee,"Angelfall (Penryn & the End of Days, #1)",2014-03-04,2016-06-23,5,"{'Fantasy': 4143, 'Young Adult': 3730, 'Parano..."
51,Jennifer L. Armentrout,"Trust in Me (Wait for You, #1.5)",2014-02-09,2014-02-13,5,"{'New Adult': 812, 'Romance': 675, 'Contempora..."
60,Jennifer L. Armentrout,"Wait for You (Wait for You, #1)",2014-02-08,2014-02-09,5,"{'New Adult': 2497, 'Romance': 2359, 'Contempo..."
62,Jennifer L. Armentrout,"Sentinel (Covenant, #5)",2014-02-06,2014-02-07,5,"{'Fantasy': 589, 'Young Adult': 572, 'Fantasy-..."


,author,title,seen,score,genre_and_votes
0,Jennifer L. Armentrout,"Opal (Lux, #3)",False,0.948367,"{'Young Adult': 1809, 'Fantasy-Paranormal': 15..."
1,Jennifer L. Armentrout,"Onyx (Lux, #2)",False,0.935156,"{'Young Adult': 2221, 'Fantasy-Paranormal': 17..."
2,Jennifer L. Armentrout,"Origin (Lux, #4)",False,0.931649,"{'Young Adult': 1423, 'Fantasy-Paranormal': 12..."
3,Jennifer L. Armentrout,"Obsidian (Lux, #1)",False,0.905284,"{'Young Adult': 3332, 'Fantasy-Paranormal': 26..."
4,Jennifer L. Armentrout,"Opposition (Lux, #5)",True,0.859427,"{'Young Adult': 1143, 'Fantasy-Paranormal': 95..."


# === Базовые подходы: контентные рекомендации

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков